# Crime Trend Line Chart

Shows 5-year crime rate trends across Toronto.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_safety` | neighbourhood × year | year, crime_rate_per_100k, crime_yoy_change_pct |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    year,
    AVG(crime_rate_per_100k) as avg_crime_rate,
    AVG(assault_rate_per_100k) as avg_assault_rate,
    AVG(auto_theft_rate_per_100k) as avg_auto_theft_rate,
    AVG(break_enter_rate_per_100k) as avg_break_enter_rate,
    SUM(total_incidents) as total_city_incidents,
    AVG(crime_yoy_change_pct) as avg_yoy_change
FROM mart_toronto.mart_neighbourhood_safety
WHERE year >= (SELECT MAX(year) - 5 FROM mart_toronto.mart_neighbourhood_safety)
GROUP BY year
ORDER BY year
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} years of crime data")

Loaded 6 years of crime data


### Transformation Steps

1. Aggregate by year (city-wide)
2. Convert year to datetime
3. Melt for multi-line by crime type

In [2]:
df["date"] = pd.to_datetime(df["year"].astype(str) + "-01-01")

# Melt for multi-line
df_melted = df.melt(
    id_vars=["year", "date"],
    value_vars=["avg_assault_rate", "avg_auto_theft_rate", "avg_break_enter_rate"],
    var_name="crime_type",
    value_name="rate_per_100k",
)

df_melted["crime_type"] = df_melted["crime_type"].map(
    {
        "avg_assault_rate": "Assault",
        "avg_auto_theft_rate": "Auto Theft",
        "avg_break_enter_rate": "Break & Enter",
    }
)

### Sample Output

In [3]:
df[["year", "avg_crime_rate", "total_city_incidents", "avg_yoy_change"]]

,year,avg_crime_rate,total_city_incidents,avg_yoy_change
0,2020,206.549993,49324.0,-4.926772
1,2021,193.839546,46377.0,-5.642532
2,2022,218.761496,53810.0,19.934873
3,2023,240.715513,61256.0,15.851392
4,2024,217.981926,179443.0,383.735380
5,2025,216.663866,3264321.0,6214.027215


## 2. Data Visualization

### Figure Factory

Uses `create_price_time_series` (reused for any numeric trend).

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.time_series import create_price_time_series

data = df_melted.to_dict("records")

fig = create_price_time_series(
    data=data,
    date_column="date",
    price_column="rate_per_100k",
    group_column="crime_type",
    title="Toronto Crime Trends by Type (5 Years)",
)

# Remove dollar sign formatting since this is rate data
fig.update_layout(yaxis_tickprefix="", yaxis_title="Rate per 100K")

fig.show()

### Overall Trend

In [5]:
# Total crime rate trend
total_data = (
    df[["date", "avg_crime_rate"]]
    .rename(columns={"avg_crime_rate": "total_rate"})
    .to_dict("records")
)

fig2 = create_price_time_series(
    data=total_data,
    date_column="date",
    price_column="total_rate",
    title="Toronto Overall Crime Rate Trend",
)
fig2.update_layout(yaxis_tickprefix="", yaxis_title="Rate per 100K")
fig2.show()